<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

# ISCE3 - L1_L_RSLC
This notebook runs the ISCE3 SLC formation SAS.

#### Kernel: isce

## Setup

First let's download the input test data and unpack it for processing:

### Now let's download that dataset from S3, but before we can we need to populate the .aws/credentials file with the access key information. 

Use a terminal to execute aws-login:

    aws-login -pub -p default -r us-west-2


In [ ]:
import os

# this block makes sure the directory set-up/change is only done once and relative to the notebook's directory
try:
    start_dir
except NameError:
    start_dir = os.getcwd()
    !mkdir -p ./notebook_output/03-L1_L_RSLC
    os.chdir('notebook_output/03-L1_L_RSLC')
    
!aws s3 cp s3://nisar-st-lts-ondemand/testdata/rslc_ALPSRP037370690.tar.gz .

In [ ]:
!tar xvfz rslc_ALPSRP037370690.tar.gz

Determine where the isce environment is installed

In [ ]:
env_str = os.popen("conda env list | grep '/isce$' | awk '{print $NF}'") # dir is always the last item in the entry
isce_base_dir = env_str.read().strip()
print(f"isce env base directory is {isce_base_dir}")

Next we disable the PGE's usage of the GPU and run the workflow:

In [ ]:
import sys,os

%cd rslc_ALPSRP037370690/

# comment out the following line if you are using GPU enabled instances
!sed -i "s/gpu_enabled: True/gpu_enabled: False/g" runconfig_rslc.yaml
!python3 -m nisar.workflows.focus runconfig_rslc.yaml
#!python $isce_base_dir/lib/python3.9/site-packages/pybind_nisar/workflows/focus.py run_config_rslc.yaml

Upon success the workflow will have created an SLC from the input L0B:

In [ ]:
!ls -l output_rslc

Let's take a look at the structure of the SLC HDF5 file:

In [ ]:
!h5ls -r output_rslc/rslc.h5

In [ ]:
!gdalinfo HDF5:"output_rslc/rslc.h5"://science/LSAR/RSLC/swaths/frequencyA/HH

In [ ]:
!gdal_translate -of ENVI HDF5:"output_rslc/rslc.h5"://science/LSAR/RSLC/swaths/frequencyA/HH HH.slc

In [ ]:
!gdalinfo HH.slc

Finally let's visualize the amplitude and phase of the SLC:

In [ ]:
import numpy as np
import osgeo.gdal as gdal
import matplotlib.pyplot as plt

ds = gdal.Open("HH.slc", gdal.GA_ReadOnly)
print (ds)
# extract a subset of the SLC to display
x0 = 0
y0 = 10
# x_offset = 1000
# y_offset = 1000
x_offset = 500
y_offset = 500

               
slc = ds.GetRasterBand(1).ReadAsArray(x0, y0, x_offset, y_offset)
print(slc)
ds = None

fig = plt.figure(figsize=(14, 12))

# display amplitude of the slc
ax = fig.add_subplot(1,2,1)
ax.imshow(np.abs(slc), vmin = -2, vmax=2, cmap='gray')
ax.set_title("amplitude")

#display phase of the slc
ax = fig.add_subplot(1,2,2)
ax.imshow(np.angle(slc))
ax.set_title("phase")

plt.show()

slc = None

<font size="1">This notebook is compatible with NISAR Jupyter Server Stack v1.7.1 and above</font>